## Ciencia de Datos Aplicada - Curso Capstone Proyecto Final

## Agrupación y Segmentación de Vecindarios en la Ciudad de Toronto, Canadá

Use el Cuaderno para crear el código para rastrear la siguiente página de Wikipedia, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, para obtener los datos que están en la tabla de códigos postales y transformar los datos en un marco de datos de pandas 

### Librerias y Dependencias

#### Se Instala librerias y Dependencias necesarias

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
#!conda install -c conda-forge geopy --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare 
from geopy.geocoders import Nominatim # convertir una dirección en valores de latitud y longitud

import requests # librería para manejar solicitudes
from pandas.io.json import json_normalize # librería para convertir un archivo json en un dataframe pandas

# Matplotlib y módulos asociados para graficar
import matplotlib.cm as cm
import matplotlib.colors as colors

# importar k-means desde la fase de agrupación
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare
import folium # librería para graficar mapas 

print('Libraries imported.')

Libraries imported.


### Web Scrapin de los Datos 

Por medio de **BeatifulSoup** se hace *Web Scraping* para obtener los datos de la página web que
contiene los códigos postales de Toronto.

In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

Una vez hecha la sopa de datos se traen solo los datos de la tabla ya que contienen los codigos
postales de Toronto.

In [4]:
table = soup.find('table')

In [5]:
file=[]
ncol=list(range(0,9))

for i in ncol :
    
    for row in table.find_all('tr'): # in html table row is represented by the table
    # Get all columns in each row.
        cols = row.find_all('p') # in html a column is represented by the tag td or p
        file.append(cols[i].getText())

    # convert to dataframe:
df= pd.DataFrame(file)

#### Tabla de Códigos Postales

Despues de traer los datos que estan contenidos en la tabla, por medio de **pandas** se le da formato a la tabla para obtener la salida deseada.

In [6]:
df["Postal Code"]=df[0].str[0:3]
df["Borough"]=df[0].str[3:].str.split("(",n=1,expand=True)[0]
df["Neigh"]=df[0].str.rsplit("(",n=2,expand=True)[1].str.split(")",n=1,expand=True)[0]
df["Neighbourhood"]=df["Neigh"].str.replace(" / ",",")

In [7]:
postalcode_list=df[(df["Borough"]!="Not assigned\n")]
postalcode_list=postalcode_list.drop([0,"Neigh"],axis=1)

In [8]:
postalcode_list.head()

,Postal Code,Borough,Neighbourhood
1,M1B,Scarborough,"Malvern,Rouge"
2,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
3,M1E,Scarborough,"Guildwood,Morningside,West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [9]:
postalcode_list.shape

(103, 3)

### Cordenadas de Latitud y Longitud

Por medio del paquete Geocoder Python: https://geocoder.readthedocs.io/index.html. Se obtienen los datos de Latitud y Longitud para cada código postal.

Para asegurarse de obtener las coordenadas de todos los vecindarios, se puede ejecutar un ciclo while para cada código postal. 

In [10]:
#"http://cocl.us/Geospatial_data"
Geospatial_Coordinates=pd.read_csv("Geospatial_Coordinates.csv")

Una vez obtenidas las coordenadas se adjuntan al dataset de códigos postales

In [11]:
coord_list=pd.merge(postalcode_list,Geospatial_Coordinates,on="Postal Code")

In [12]:
coord_list.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Agrupación y Segmentación

A continuación vamos a empezar a utilizar el API de FourSquare para explorar los barrios y segmentarlos.

#### Mapa de los barrios de Toronto

Se va a generar un mapa de Toronto con todos sus barrios, para iniciar el análisis de segmentación

Utilizando la libreria geopy, se obtiene las coordenadas de Toronto. Para poder definir una instancia del geocoder necesitaremos definir un user_agent. Nombraremos a nuestro agente ny_explorer, como se muestra a continuación.

In [13]:
address = 'TORONTO,CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Para el ejecicio de eligirán los barrios que contienen la palabra Toronto

In [14]:
toronto_data = coord_list[coord_list['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.groupby("Borough").size().sort_values(ascending=False)

Borough
Downtown Toronto                                                17
Central Toronto                                                  9
West Toronto                                                     6
East Toronto                                                     4
Downtown TorontoStn A PO Boxes25 The Esplanade                   1
East TorontoBusiness reply mail Processing Centre969 Eastern     1
East YorkEast Toronto                                            1
dtype: int64

Se ubican los barrios de Toronto en un mapa

In [15]:
# crear un mapa de Manhattan usando los valores de latitud y longitud
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# añadir los marcadores al mapa
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Definir version y credenciales de Foursquare
A continuación vamos a empezar a utilizar el API de FourSquare para explorar los barrios y segmentarlos, por medio de una función que explore todos los barrios.

In [16]:
CLIENT_ID = 'LM5HN3042JGPC50VLXUT2XG4L1Z4BN3DY1S5MBA5ANSVO54V' # su ID de Foursquare
CLIENT_SECRET = 'VXEFAC1AOKO15P0VCDGWBUV0QPB4HCSV1YFEHGG4KZWXTAHS' # Secreto de Foursquare
VERSION = '20180605' # versión de la API de Foursquare
LIMIT = 100 # Un valor límite para la API de Foursquare

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LM5HN3042JGPC50VLXUT2XG4L1Z4BN3DY1S5MBA5ANSVO54V
CLIENT_SECRET:VXEFAC1AOKO15P0VCDGWBUV0QPB4HCSV1YFEHGG4KZWXTAHS


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # crear la URL de solicitud de API
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # solicitud GET
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # regresa solo información relevante de cada sitio cercano
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

De la función se obtienen todas las calles de toronto

In [21]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth  East
The Danforth West,Riverdale
India Bazaar,The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Summerhill West,Rathnelly,South Hill,Forest Hill SE,Deer Park
Rosedale
St. James Town,Cabbagetown
Church and Wellesley
Regent Park,Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond,Adelaide,King
Harbourfront East,Union Station,Toronto Islands
Toronto Dominion Centre,Design Exchange
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North & West
The Annex,North Midtown,Yorkville
University of Toronto,Harbord
Kensington Market,Chinatown,Grange Park
CN Tower,King and Spadina,Railway Lands,Harbourfront West,Bathurst Quay,South Niagara,Island airport
Enclave of M5E
First Canadian Place,Underground city
Christie
Dufferin,Dovercourt Village
Little Portugal,Trinity
Brockton,Parkdale Village,Exhibition Place
High Park,The Junction South
Parkdale,Roncesvalles
Runn

In [23]:
print(toronto_venues.shape)
toronto_venues.head()

(1603, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth East,43.685347,-79.338106,The Path,43.683923,-79.335007,Park


Explorando los sitios de cada barrio

In [26]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton,Parkdale Village,Exhibition Place",24,24,24,24,24,24
"CN Tower,King and Spadina,Railway Lands,Harbourfront West,Bathurst Quay,South Niagara,Island airport",15,15,15,15,15,15
Central Bay Street,65,65,65,65,65,65
Christie,16,16,16,16,16,16
Church and Wellesley,83,83,83,83,83,83
"Commerce Court,Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34
Davisville North,9,9,9,9,9,9


Categorías únicas se pueden conservar de todos los sitios regresados

In [29]:
print('Hay {} categorias únicas.'.format(len(toronto_venues['Venue Category'].unique())))

Hay 225 categorias únicas.


#### Analizando cada Barrio

In [31]:
# codificación
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# añadir la columna de barrio de regreso al dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# mover la columna de barrio a la primer columna
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Danforth East,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.017544,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000
1,"Brockton,Parkdale Village,Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.041667
2,"CN Tower,King and Spadina,Railway Lands,Harbou...",0.066667,0.066667,0.066667,0.133333,0.2,0.066667,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.015385,0.000000,0.000000,0.000000,0.015385
4,Christie,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.012048,0.000000,0.00,...,0.012048,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.024096
6,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.030000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.000000,0.010000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.029412,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Dufferin,Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


Cada barrio junto con los 5 sitios mas comunes

In [36]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0          Restaurant  0.05
1         Coffee Shop  0.05
2            Beer Bar  0.04
3  Seafood Restaurant  0.04
4      Farmers Market  0.04


----Brockton,Parkdale Village,Exhibition Place----
            venue  freq
0            Café  0.12
1          Bakery  0.08
2  Breakfast Spot  0.08
3     Coffee Shop  0.08
4             Gym  0.04


----CN Tower,King and Spadina,Railway Lands,Harbourfront West,Bathurst Quay,South Niagara,Island airport----
                 venue  freq
0      Airport Service  0.20
1       Airport Lounge  0.13
2              Airport  0.07
3   Airport Food Court  0.07
4  Rental Car Location  0.07


----Central Bay Street----
            venue  freq
0     Coffee Shop  0.20
1  Sandwich Place  0.08
2            Café  0.06
3      Restaurant  0.05
4     Salad Place  0.03


----Christie----
           venue  freq
0  Grocery Store  0.19
1           Café  0.19
2    Coffee Shop  0.12
3           Park  0.12
4     Restaurant  0.06


Primero escribamos una función para ordenar los sitios en orden descendente.

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Generemos el nuevo dataframe y mostremos los primeros 10 sitios de cada barrio.

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# crear las columnas acorde al numero de sitios populares
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# crear un nuevo dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Restaurant,Coffee Shop,Beer Bar,Seafood Restaurant,Farmers Market,Pharmacy,Bakery,Cocktail Bar,Cheese Shop,Comfort Food Restaurant
1,"Brockton,Parkdale Village,Exhibition Place",Café,Bakery,Breakfast Spot,Coffee Shop,Gym,Burrito Place,Climbing Gym,Convenience Store,Furniture / Home Store,Grocery Store
2,"CN Tower,King and Spadina,Railway Lands,Harbou...",Airport Service,Airport Lounge,Airport,Airport Food Court,Rental Car Location,Sculpture Garden,Boat or Ferry,Bar,Boutique,Airport Gate
3,Central Bay Street,Coffee Shop,Sandwich Place,Café,Restaurant,Salad Place,Department Store,Spa,Italian Restaurant,Burger Joint,Park
4,Christie,Grocery Store,Café,Coffee Shop,Park,Restaurant,Japanese Restaurant,Nightclub,Athletics & Sports,Baby Store,Candy Store


### Modelo De Clustering

Se ejecuta el modelo de _k_-means para agrupar los barrios en 5 grupos también llamado anális de clustering.

In [39]:
# establecer el número de agrupaciones
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# ejecutar k-means
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# revisar las etiquetas de las agrupaciones generadas para cada fila del dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

Generemos un nuevo dataframe que incluya la agrupación asi como los 10 sitios mas populares de cada barrio.

In [40]:
# añadir etiquetas
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# juntar manhattan_grouped con manhattan_data 
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # revisar las ultimas columnas

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0.0,Health Food Store,Trail,Pub,Neighborhood,Airport,Museum,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
1,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106,3.0,Convenience Store,Park,Airport,Museum,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
2,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,2.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Yoga Studio,Brewery,Fruit & Vegetable Store,Frozen Yogurt Shop,Bookstore
3,M4L,East Toronto,"India Bazaar,The Beaches West",43.668999,-79.315572,2.0,Fast Food Restaurant,Park,Italian Restaurant,Brewery,Burrito Place,Fish & Chips Shop,Food & Drink Shop,Liquor Store,Sandwich Place,Steakhouse
4,M4M,East Toronto,Studio District,43.659526,-79.340923,2.0,Coffee Shop,Brewery,Café,American Restaurant,Gastropub,Bakery,Yoga Studio,Diner,Cheese Shop,Seafood Restaurant


Finalmente visualicemos las agrupaciones resultantes

In [78]:
# crear mapa
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# establecer el esquema de color para las agrupaciones
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# añadir marcadores al mapa
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        fill_color="blue",#rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Análisis de los Cluster

Ahora se hace un análisis de las agrupaciones y determinar las categorias del sitio que distingue a cada agrupación. En base a las categorias definidas se asignará un nombre a cada agrupación. 

#### Agrupación 1. Este

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0.0,Health Food Store,Trail,Pub,Neighborhood,Airport,Museum,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant


#### Agrupación 2. Centro

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,1.0,Garden,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant


Agrupación 3. Downtown

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,2.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Yoga Studio,Brewery,Fruit & Vegetable Store,Frozen Yogurt Shop,Bookstore
3,East Toronto,2.0,Fast Food Restaurant,Park,Italian Restaurant,Brewery,Burrito Place,Fish & Chips Shop,Food & Drink Shop,Liquor Store,Sandwich Place,Steakhouse
4,East Toronto,2.0,Coffee Shop,Brewery,Café,American Restaurant,Gastropub,Bakery,Yoga Studio,Diner,Cheese Shop,Seafood Restaurant
6,Central Toronto,2.0,Gym,Breakfast Spot,Hotel,Food & Drink Shop,Department Store,Park,Sandwich Place,Pizza Place,Gym / Fitness Center,Lounge
7,Central Toronto,2.0,Coffee Shop,Clothing Store,Yoga Studio,Furniture / Home Store,Diner,Restaurant,Chinese Restaurant,Salon / Barbershop,Café,Shoe Store
8,Central Toronto,2.0,Dessert Shop,Sandwich Place,Gym,Thai Restaurant,Italian Restaurant,Pizza Place,Coffee Shop,Sushi Restaurant,Café,Brewery
10,Central Toronto,2.0,Coffee Shop,Liquor Store,Pub,Light Rail Station,Fried Chicken Joint,Supermarket,Sushi Restaurant,Bank,Restaurant,Bagel Shop
12,Downtown Toronto,2.0,Pizza Place,Café,Coffee Shop,Italian Restaurant,Bakery,Pub,Restaurant,Convenience Store,Chinese Restaurant,Liquor Store
13,Downtown Toronto,2.0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Pizza Place,Restaurant,Pub,Café,Fast Food Restaurant,Bubble Tea Shop
14,Downtown Toronto,2.0,Coffee Shop,Park,Café,Bakery,Pub,Breakfast Spot,Theater,Spa,Shoe Store,Chocolate Shop


#### Agrupación 5. Este Suburbios

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East YorkEast Toronto,3.0,Convenience Store,Park,Airport,Museum,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


#### Agrupación 5. Parques

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,4.0,Park,Bus Line,Swim School,Airport,Movie Theater,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
11,Downtown Toronto,4.0,Park,Playground,Trail,Airport,Museum,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
24,Central Toronto,4.0,Park,Trail,Bus Line,Jewelry Store,Sushi Restaurant,Airport,Movie Theater,Men's Store,Metro Station,Mexican Restaurant
